In [9]:
text = ''
words = [word.strip() for word in text.split(',')]
print(words)


['']


In [13]:
list1 = ['aa', 'ec','bbbb']
list2 = ['aaaa', 'bbbb', 'aa']

# 查找 list1 中的每个字符串在 list2 中的索引位置，并生成集合
positions = {i for i, word in enumerate(list2) if word in list1}

print(positions)

{1, 2}


In [6]:
words

['roomname,total_contract_price']

In [1]:
import psycopg2
import pymysql
import os
import pandas as pd
import json
from datetime import date
import re
import traceback
from decimal import Decimal, ROUND_HALF_UP
from openai import OpenAI
import Levenshtein  # 使用 Levenshtein 库来计算字符串距离
import time
from utils import default_converter,is_chinese,extract_json_from_response,format_decimal_value

In [2]:
connection = psycopg2.connect(
    dbname="fdc_dc",
    user="dws_user_hwai",
    password="NewHope#1982@",
    host="124.70.57.67",
    port="8000",
)
connection.set_client_encoding('UTF8')

In [3]:
sql_query = "SELECT * FROM fdc_dwd.dwd_trade_roomsign_a_min WHERE projname = '成都锦官阁' LIMIT 15;"

In [4]:
cursor = connection.cursor()
start_time = time.time()
cursor.execute(sql_query)

results = cursor.fetchall()
end_time = time.time()
elapsed_time = end_time - start_time
print(f"查询耗时{elapsed_time}秒")

查询耗时12.3017418384552秒


In [5]:
column_description = cursor.description

In [6]:
column_names, type_codes = zip(*((des[0], des[1]) for des in column_description))

In [7]:
with open("en2ch.json", "r", encoding="utf-8") as file:
    en2zh_json = json.load(file)

In [8]:
def get_translate_column_names(column_names):
    need_translate_list = []
    translated_column_names = []
    for column in column_names:
        if column not in en2zh_json.keys() and not is_chinese(column):
            need_translate_list.append(column)
    if need_translate_list == []:
        for column in column_names:
            if column in en2zh_json.keys():
                translated_column_names.append(en2zh_json[column])
            else:
                translated_column_names.append(column)
        return translated_column_names

    prompt_template = """
    请帮我将以下变量名从英文翻译为简洁的中文变量名。变量名会以Python列表的形式提供，如：['avg_daily_outbound', 'days_to_deplete']。请注意，翻译时应尽可能保持变量名原有的语义，并以简洁准确的中文形式表示。结果应以JSON格式返回，其中英文变量名作为键，中文翻译作为值。例如：
    输入：['avg_daily_outbound', 'days_to_deplete']
    输出：{
    "avg_daily_outbound": "日均出库量",
    "days_to_deplete": "耗尽天数"
    }

    以下是需要翻译的变量列表：
    <input>
    """
    final_prompt = prompt_template.replace("<input>", str(need_translate_list))
    client = OpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    response = client.chat.completions.create(
        model="qwen2.5-72b-instruct",
        messages=[{"role": "user", "content": final_prompt}],
    )
    temp_dict = extract_json_from_response(response.choices[0].message.content)

    for column in column_names:
        if column not in en2zh_json.keys() and not is_chinese(column):
            translated_column_names.append(temp_dict[column])
        elif column in en2zh_json.keys():
            translated_column_names.append(en2zh_json[column])
        else:
            translated_column_names.append(column)
    return translated_column_names

In [9]:
translated_column_names = get_translate_column_names(column_names)

In [12]:
translated_column_names,type_codes, results

(['房间编码',
  '房间名称',
  '房间全称',
  '楼栋编码',
  '楼栋名称',
  '楼栋全称',
  '分期编码',
  '分期名称',
  '项目编码',
  '项目名称',
  '业态编码',
  '业态全称',
  '业态名称',
  '城市编码',
  '城市名称',
  '法人公司编码',
  '法人公司名称',
  '置业顾问oaAccount',
  '置业顾问名称',
  '合同编码',
  '合同号',
  '销售单编码',
  '来访编码',
  '认购编码',
  '客户编码',
  '客户名称',
  '客户类型',
  '计价方式',
  '付款方式',
  '付款方式类别',
  '合同备案号',
  '签约日期',
  '认购日期',
  '网签日期',
  '备案日期',
  '合同状态',
  '面积状态',
  '建筑面积',
  '套内面积',
  '建筑单价',
  '套内单价',
  '建筑成交单价',
  '套内成交单价',
  '合同总价',
  '装修付款方式',
  '装修款是否并入合同',
  '装修标准',
  '装修单价',
  '初装修改造费',
  '装修总价',
  '附属款',
  '按揭银行',
  '按揭贷款额',
  '按揭年限',
  '公积金银行',
  '公积金贷款额',
  '公积金年限',
  '折扣',
  '折扣说明',
  '推荐人',
  '业务员oa号',
  '业务员',
  '关闭日期',
  '关闭原因',
  '修改日期',
  '客户来源',
  '是否首付分期',
  '签约统计日期',
  '实际补差款',
  '补差款审核时间',
  '溢价款',
  '分区日期',
  '置业顾问ids,可能包含多个置业顾问',
  '标准总价',
  '按揭记录',
  '变更前合同编码',
  '按揭银行',
  '返现'],
 (1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  1043,
  10

In [26]:
len(results)

15

In [19]:
response_data = {col: [] for col in translated_column_names}
for row in results:
    for col_name, value in zip(translated_column_names, row):
        response_data[col_name].append(format_decimal_value(value))
response_data

{'房间编码': ['34859FCD-044C-E711-9401-90B11C25CCB5',
  '15C0C8CC-97B4-E711-940B-90B11C25CCB5',
  '1BFE88EF-A8DC-E711-940D-90B11C25CCB3',
  'AFE72087-044C-E711-9401-90B11C25CCB5',
  '9D57E6BA-8F78-E611-93F5-90B11C25CCB5',
  'FFBFC8CC-97B4-E711-940B-90B11C25CCB5',
  '9C57E6BA-8F78-E611-93F5-90B11C25CCB5',
  'A3C0C8CC-97B4-E711-940B-90B11C25CCB5',
  'DF7404A8-A8B4-E711-940B-90B11C25CCB5',
  '4BC1C8CC-97B4-E711-940B-90B11C25CCB5',
  '497504A8-A8B4-E711-940B-90B11C25CCB5',
  'A87404A8-A8B4-E711-940B-90B11C25CCB5',
  'AABFC8CC-97B4-E711-940B-90B11C25CCB5',
  'E1EDB0E6-ADB4-E711-940B-90B11C25CCB5',
  'A7E72087-044C-E711-9401-90B11C25CCB5'],
 '房间名称': ['成都锦官阁-1期-1栋商业-113',
  '成都锦官阁-1期-普通车位-313',
  '成都锦官阁-1期-3底商-负104',
  '成都锦官阁-1期-2栋商业-105',
  '成都锦官阁-1期-4栋-1-303',
  '成都锦官阁-1期-普通车位-490',
  '成都锦官阁-1期-4栋-1-302',
  '成都锦官阁-1期-普通车位-256',
  '成都锦官阁-1期-普通车位-864',
  '成都锦官阁-1期-普通车位-026',
  '成都锦官阁-1期-普通车位-771',
  '成都锦官阁-1期-普通车位-809',
  '成都锦官阁-1期-普通车位-405',
  '成都锦官阁-1期-负二层人防车位-576',
  '成都锦官阁-1期-2栋商业-208'],
 '房间

In [23]:
response_columns = []
for i in range(len(translated_column_names)):
    # 如果是数值型
    if type_codes[i] == 1700:
        numbers = [x for x in response_data[translated_column_names[i]] if x is not None]
        if numbers == []:
            response_columns.append({"name":translated_column_names[i],"field_type":"指标","default_display":True,"stats":{}})
        else:
            total_sum = sum(numbers)
            stats = {"sum":total_sum,"avg":total_sum / len(numbers) if numbers else 0,"max":max(numbers),"min":min(numbers)}
            response_columns.append({"name":translated_column_names[i],"field_type":"指标","default_display":True,"stats":stats})
    else:
        response_columns.append({"name":translated_column_names[i],"field_type":"维度","default_display":True})

In [24]:
response_columns

[{'name': '房间编码', 'field_type': '维度', 'default_display': True},
 {'name': '房间名称', 'field_type': '维度', 'default_display': True},
 {'name': '房间全称', 'field_type': '维度', 'default_display': True},
 {'name': '楼栋编码', 'field_type': '维度', 'default_display': True},
 {'name': '楼栋名称', 'field_type': '维度', 'default_display': True},
 {'name': '楼栋全称', 'field_type': '维度', 'default_display': True},
 {'name': '分期编码', 'field_type': '维度', 'default_display': True},
 {'name': '分期名称', 'field_type': '维度', 'default_display': True},
 {'name': '项目编码', 'field_type': '维度', 'default_display': True},
 {'name': '项目名称', 'field_type': '维度', 'default_display': True},
 {'name': '业态编码', 'field_type': '维度', 'default_display': True},
 {'name': '业态全称', 'field_type': '维度', 'default_display': True},
 {'name': '业态名称', 'field_type': '维度', 'default_display': True},
 {'name': '城市编码', 'field_type': '维度', 'default_display': True},
 {'name': '城市名称', 'field_type': '维度', 'default_display': True},
 {'name': '法人公司编码', 'field_type': '维度', 

In [ ]:
response_metadata = {"sql": "SELECT column1, column2, column3 FROM table_name","record_count":len(results),"display_type": "response_table"}

In [ ]:
sql_response = {"columns":response_columns,"data":response_data,"metadata":response_metadata}